# GraphFlood: Local inputs

This example shows how to use vanilla Graphflood with discrete localised source of water.
We use as input a DEM displaying a river flowing from South to North. It involves setting up boundary conditions, and localise water inflow.

## Setting up the model structure

In [ ]:
import scabbard as scb
import dagger as dag
import numpy as np
import matplotlib.pyplot as plt
import math as m
import matplotlib
import random
from scipy.ndimage import gaussian_filter
from scabbard import ModelHelper, PlotHelper
%matplotlib widget

# Name of the DEM to load
fnamedem = "dem_section.tif"

# Graphflood object
mod = ModelHelper()

# precipitation rates in m/s
# Setting it to no precipitation
P = 0.

# initialising model
mod.init_dem_model(fnamedem, sea_level = 0., P = P)

# #IMPORTANT, in a cell bellow, I explain that boundary conditions can be a problem when flow manages to output the model close to where flow is inputted
# # Uncomment these to force the code to only being able to output at the North boundary, to block all the E/W boundaries and to force the S to only input flow (cannot leave the model From there)
# ## Initialising boundary to all normal nodes (this will crash if you leave the edges as normal nodes)
# BCs = np.ones((mod.grid.ny,mod.grid.nx), dtype = np.uint8) # -> note the dtype is unsigned integer 8 bits
# ## Setting all the nodes of the left and right columns to no data 
# BCs[:, [0,-1]] = 0
# ## first Row, North, to only outflow
# BCs[0,:] = 3
# ## last Row, South, to only inflow
# BCs[-1,:] = 7
# ## Reinitialise the model with new BCS -> be careful to do that before setting other parameter otherwise they might get overwritten
# mod.init_dem_model(fnamedem, sea_level = 0., P = P, BCs = BCs)



# Use CFL conditions to adapt timestep
mod.courant = False
mod.min_courant_dt = 1e-6 # Minimum timestep if CFL dt
mod.courant_number = 0.04 # Courant number modulates dt: higher courant = higher dt. Depends on many things (dx, P, ...)

# if courant is False, this will be used as time step
mod.dt = 1e-3

# Stationary = run to equillibrium
# Stationary = False propagate a trasient flood wave (the model has not been developped for that though)
mod.stationary = True 

# manning friction coeff, 0.033 is an OK value for open flow in channels:
mod.mannings = 0.033

# Single flow solver or Multiple flow solver?
# SFD: faster, less accurate
# MFD: slower, more accurate and prettier
mod.SFD = False

## Setting the water influx

To set the water flux, we need to identify nodes where we will input water.
We will simply plot the DEM and find their row/col index. Here we want to add them in the South (last row so -1)


In [ ]:
fig,ax = plt.subplots()
ax.imshow(mod.grid.Z2D, cmap = 'terrain')
ax.imshow(mod.grid.hillshade, cmap = 'gray', alpha = 0.4)

In [ ]:
# creating a 1D vectorised indices and reshaping it to 2D
indices = np.arange(mod.grid.nx * mod.grid.ny).reshape(mod.grid.rshp)

# creating an array of node where you want to input water (array[row_idx,col_idx] will give the node index of the required node)
# to get these I recommend plotting an imshow in a separate cell (like above) without the extent keyword then you can use the widget to zoom on the node you need and input their row (y starting from the top) and col (x starting from the left):
# Here, I am selecting all the nodes of columns 80 to 90 of the last row
input_nodes = np.array([
    indices[-1,80:90],
]).astype(np.int32) # that last bit is just to ensure the data is in the right format -> 32 bits signed integer

# Total discharge to input to the model via that boundary
total_Qw_in = 5 # m^3/s

# Spreading that discharge uniformally over all the pixels. Alternatively you can manually set a discharge per pixel as long as input_values as the same dimension than total_Qw_in
input_values = total_Qw_in/input_nodes.shape[0] * np.ones_like(input_nodes).astype(np.float64)

# Important: model accept values as flow depth per second. So your volumetric discharge divided by the cell area
input_values /= mod.grid.dx*mod.grid.dy

# Adding the flow to the model
mod.gf.flood.set_water_input_by_entry_points(input_values,input_nodes)

# IMPORTANT CONSIDERATION: the nodes in input_nodes can only input flow which cannot leave the model through these boundaries
# In case you want to block boundaries, one can hackily block flow with artificial walls by modifying the raster
# A cleaner way, yet more complicated means changing boundary conditions. Each pixel in the model has a boundary code: 0 = no data, 1 = normal internal node, 3 = can_out and 7 = input (there are many others but let's keep to them)
# See the first cell for an example of the latter


mod.gf.flood.set_water_input_by_entry_points(input_values,input_nodes)

# Plot a dynamic figure (also works outside of jupyter)
ph = PlotHelper(mod, jupyter = True)
ph.init_hw_plot(use_extent = True, vmax = 1.5)

In [ ]:
# Update the above figure every X timesteps
update_fig = 20
# Total number of run for this time step
Ndt = 10000
for i in range(Ndt):
    # Run one dt
    mod.run()
    if(i % update_fig > 0):
        continue
    print(i, end = '\r')
    # Update figure
    ph.update()

## Accessing the data

In [ ]:
# Access data:
## Flow depth
hw = mod.hw
## Discharge in (needs reshaping in 2D)
Qwi = mod.gf.flood.get_Qwin().reshape(mod.grid.rshp)
## Discharge out (needs reshaping in 2D) -> Should be roughly equal to Qwin if the model converges correctly as we made the steady-flow assumption
Qwo = mod.gf.flood.compute_tuqQ(3).reshape(mod.grid.rshp)
## Discharge per unit width:
qw = mod.gf.flood.compute_tuqQ(2).reshape(mod.grid.rshp)
## flow velocity:
uw = mod.gf.flood.compute_tuqQ(1).reshape(mod.grid.rshp)
 

### Quick example to plot them

In [ ]:
fig,ax = plt.subplots()
im = ax.imshow(uw, cmap = 'magma', extent = mod.grid.extent())
plt.colorbar(im, label = r'$\vec{u} ~ (m.s^{-1})$')
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')


Data are sets of numpy 2D array, you can export/save them as such.


In case you need georeferenced data it is possible as you just "replace" the field of value from yur original DEM with these (use rasterio for example)